In [1]:
import re

#### Cell 1: Определение паттернов для токенизации
Определяем паттерны для токенов, используя регулярные выражения. Это включает скобки, операторы, числа и переменные.

In [2]:
patterns = [
    (re.compile(r'\('), 'LPAREN'),
    (re.compile(r'\)'), 'RPAREN'),
    (re.compile(r'\+'), 'PLUS'),
    (re.compile(r'\-'), 'MINUS'),
    (re.compile(r'\*\*'), 'POWER'),
    (re.compile(r'\*'), 'MULTIPLY'),
    (re.compile(r'\/'), 'DIVIDE'),
    (re.compile(r'\d+'), 'NUMBER'),
    (re.compile(r'lambda'), 'LAMBDA'),
    (re.compile(r'λ'), 'LAMBDA'),
    (re.compile(r'\.'), 'DOT'),
    (re.compile(r'\,'), 'COMMA'),
    (re.compile(r'\:'), 'COLON'),    
    (re.compile(r'[a-zA-Z][a-zA-Z0-9]*'), 'VARIABLE'),
]

#### Cell 2: Приоритеты операторов
Задаём словарь `OPERATOR_PRIORITY` для определения приоритета операторов, что важно для правильного построения AST.

In [3]:
OPERATOR_PRIORITY = {
    'POWER': 4,
    'MULTIPLY': 3,
    'DIVIDE': 3,
    'PLUS': 2,
    'MINUS': 2,
}

#### Cell 3: Функция токенизации
Реализуем функцию `tokenize`, преобразующую входную строку в список токенов на основе предопределённых паттернов.

In [4]:
def tokenize(input_string):
    tokens = []
    position = 0
    while position < len(input_string):
        match = None
        for pattern, token_type in patterns:
            regex_match = pattern.match(input_string, position)
            if regex_match:
                match = regex_match.group(0)
                tokens.append((token_type, match))
                position += len(match)
                break
        if not match:
            # Не найдено соответствий: пропускаем символ
            position += 1
    return tokens


#### Cell 4: Тестирование токенизации
Определяем тестовое лямбда-выражение и применяем функцию токенизации для получения списка токенов.

In [5]:
true_expr = lambda x, y: x**2 - (x * 3 + y)
lambda_expr = 'lambda x, y: x**2 - (x * 3 + y)'
tokens = tokenize(lambda_expr)
print(tokens)

[('LAMBDA', 'lambda'), ('VARIABLE', 'x'), ('COMMA', ','), ('VARIABLE', 'y'), ('COLON', ':'), ('VARIABLE', 'x'), ('POWER', '**'), ('NUMBER', '2'), ('MINUS', '-'), ('LPAREN', '('), ('VARIABLE', 'x'), ('MULTIPLY', '*'), ('NUMBER', '3'), ('PLUS', '+'), ('VARIABLE', 'y'), ('RPAREN', ')')]


#### Cell 5: Определение структуры узла
Определяем класс `Node` для представления узлов в AST, каждый узел может иметь тип, значение и дочерние узлы.

In [6]:
class Node:
    def __init__(self, type, value=None, children=None):
        self.type = type
        self.value = value
        self.children = children if children else []

    def __repr__(self):
        return self.pretty_print()

    def pretty_print(self, level=0):
        ret = "    " * level + f"{self.type}: {self.value}\n"
        for child in self.children:
            ret += child.pretty_print(level + 1)
        return ret

#### Cell 6: Парсинг выражения в AST
Реализуем функцию `parse_expression`, которая преобразует список токенов в AST, учитывая приоритеты операций.

In [7]:
def parse_expression(tokens, start=0, end=None):
    if end is None:
        end = len(tokens)
    
    if start == end:
        return None
    elif end - start == 1:
        token = tokens[start]
        return Node(token[0], value=token[1])

    # Находим основную операцию (пропускаем выражения в скобках)
    min_priority = float('inf')
    main_op = None
    i = start
    while i < end:
        token = tokens[i]
        if token[0] == 'LPAREN':
            # Пропускаем содержимое скобок
            balance = 1
            while balance != 0:
                i += 1
                if tokens[i][0] == 'LPAREN':
                    balance += 1
                elif tokens[i][0] == 'RPAREN':
                    balance -= 1
        elif token[0] in ['PLUS', 'MINUS', 'MULTIPLY', 'POWER'] and OPERATOR_PRIORITY[token[0]] <= min_priority:
            min_priority = OPERATOR_PRIORITY[token[0]]
            main_op = i
        i += 1

    if main_op is not None:
        left = parse_expression(tokens, start, main_op)
        right = parse_expression(tokens, main_op + 1, end)
        return Node('OPERATION', value=tokens[main_op][1], children=[left, right])
    
    if tokens[start][0] == 'LPAREN' and tokens[end-1][0] == 'RPAREN':
        return parse_expression(tokens, start + 1, end - 1)

    # Необходим дополнительный анализ для других случаев
    raise NotImplementedError("Parser logic for other cases")

def build_ast(tokens):
    # Убеждаемся, что первый токен - 'LAMBDA'
    assert tokens[0][0] == 'LAMBDA'
    
    i = 1  # Начинаем с первого токена после 'LAMBDA'
    params = []
    while tokens[i][0] != 'COLON':
        # Собираем все параметры до двоеточия
        if tokens[i][0] == 'VARIABLE':
            params.append(tokens[i][1])
        i += 1

    # Тело лямбда-выражения начинается после ':'
    body_start = i + 1
    body_node = parse_expression(tokens[body_start:])

    # Создаем узел лямбда-выражения с параметрами и телом
    return Node('LAMBDA', value=params, children=[body_node])

#### Cell 7: Визуализация AST
Строим AST из токенов и выводим его структуру для визуальной проверки корректности парсинга.

In [8]:
ast = build_ast(tokens)
print(lambda_expr)
print(ast)

lambda x, y: x**2 - (x * 3 + y)
LAMBDA: ['x', 'y']
    OPERATION: -
        OPERATION: **
            VARIABLE: x
            NUMBER: 2
        OPERATION: +
            OPERATION: *
                VARIABLE: x
                NUMBER: 3
            VARIABLE: y


#### Cell 8: Вычисление значения AST
Реализуем функцию `evaluate_ast`, которая вычисляет результат выражения, представленного в виде AST, для заданных значений переменных.

In [9]:
def evaluate_ast(node, variables=None):
    if variables is None:
        variables = {}

    if node.type == 'NUMBER':
        return float(node.value)
    elif node.type == 'VARIABLE':
        # Здесь используем значения переменных из словаря
        if node.value in variables:
            return variables[node.value]
        else:
            raise ValueError(f"Undefined variable: {node.value}")
    elif node.type == 'OPERATION':
        if node.value == '+':
            return evaluate_ast(node.children[0], variables) + evaluate_ast(node.children[1], variables)
        elif node.value == '-':
            return evaluate_ast(node.children[0], variables) - evaluate_ast(node.children[1], variables)
        elif node.value == '*':
            return evaluate_ast(node.children[0], variables) * evaluate_ast(node.children[1], variables)
        elif node.value == '**':
            return evaluate_ast(node.children[0], variables) ** evaluate_ast(node.children[1], variables)
    elif node.type == 'LAMBDA':
        return evaluate_ast(node.children[0], variables)
    else:
        raise ValueError(f"Unknown node type: {node.type}")

#### Cell 9: Тестирование вычисления выражения
Тестируем вычисление AST с конкретными значениями переменных и сравниваем результат с ожидаемым значением.

In [10]:
x = 2
y = 3
result = evaluate_ast(ast, {'x' : x, 'y' : y})
print('parsed result = ', result)
print('true result = ', true_expr(x, y))

parsed result =  -5.0
true result =  -5
